### Decision Trees

In [15]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
# from sklearn.tree import export_graphviz
# from sklearn.externals.six import StringIO 
# from IPython.display import Image 
# from pydot import graph_from_dot_data
import numpy as np

In [2]:
data = pd.read_csv("agaricus-lepiota.data", header = None)

In [3]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [4]:
# changing index cols with rename() 
data = data.rename(columns = {"0": "classes",  
                                "1": "cap_shape",
                                "2": "cap_surface",
                                "3": "cap_color",
                                "4": "bruises",
                                "5": "odor",
                                "6": "gill_attachment",
                                "7": "gill_spacing",
                                "8": "gill_size",
                                "9": "gill_color",
                                "10": "stalk_shape",
                                "11": "stalk_root",
                                "12": "stalk_surface_above_ring",
                                "13": "stalk_surface_below_ring",
                                "14": "stalk_color_above_ring",
                                "15": "stalk_color_below_ring",
                                "16": "veil_type",
                                "17": "veil_color",
                                "18": "ring_number",
                                "19": "ring_type",
                                "20": "spore_print_color",
                                "21": "population",
                                "22": "habitat"})

# changing columns using .columns() 
data.columns = ["classes", 
                "cap_shape", 
                "cap_surface",
                "cap_color",
                "bruises", 
                "odor", 
                "gill_attachment", 
                "gill_spacing", 
                "gill_size", 
                "gill_color", 
                "stalk_shape", 
                "stalk_root",
                "stalk_surface_above_ring", 
                "stalk_surface_below_ring", 
                "stalk_color_above_ring", 
                "stalk_color_below_ring", 
                "veil_type", 
                "veil_color", 
                "ring_number", 
                "ring_type", 
                "spore_print_color", 
                "population", 
                "habitat"]
data.columns

Index(['classes', 'cap_shape', 'cap_surface', 'cap_color', 'bruises', 'odor',
       'gill_attachment', 'gill_spacing', 'gill_size', 'gill_color',
       'stalk_shape', 'stalk_root', 'stalk_surface_above_ring',
       'stalk_surface_below_ring', 'stalk_color_above_ring',
       'stalk_color_below_ring', 'veil_type', 'veil_color', 'ring_number',
       'ring_type', 'spore_print_color', 'population', 'habitat'],
      dtype='object')

In [5]:
data['classes']= ['poisonous' if each == 'p' else 'edible' for each in data['classes']]
data['cap_shape'] = data['cap_shape'].map({'x': "convex", 'b': "bell", 's': "sunken", 'f': "flat", 'k': "knobbed", 'c': "conical"})
data['cap_surface'] = data['cap_surface'].map({'s': "smooth", 'y': "scaly", 'f': "fibrous", 'g': "grooves"})
data['cap_color'] = data['cap_color'].map({'n': "brown", 'y': "yellow", 'w': "white", 'g': "gray", 'e': "red", 'p': "pink", 'b': "buff", 'u': "purple", 'c': "cinnamon", 'r': "green"})
data['bruises']= ['bruises' if each == 't' else 'no' for each in data['bruises']]
data['odor'] = data['odor'].map({'p': "pungent", 'a': "almond", 'l': "anise", 'n': "none", 'f': "foul", 'c': "creosote", 'y': "fishy", 's': "spicy", 'm': "musty"})
data['gill_attachment'] = data['gill_attachment'].map({'f': "free", 'a': "attached", 'd': "descending", 'n': "notched"})
data['gill_spacing'] = data['gill_spacing'].map({'c': "close", 'w': "crowded", 'd': "distant"})
data['gill_size']= ['narrow' if each == 'n' else 'broad' for each in data['gill_size']]
data['gill_color'] = data['gill_color'].map({'k': "black", 'n': "brown", 'g': "gray", 'p': "pink", 'w': "white", 'h': "chocolate", 'u': "purple", 'e': "red", 'b': "buff", 'r': "green", 'y': "yellow", 'o': "orange"})
data['stalk_shape']= ['enlarging' if each == 'e' else 'tapering' for each in data['stalk_shape']]
data['stalk_root'] = data['stalk_root'].map({'e': "equal", 'c': "club", 'b': "bulbous", 'r': "rooted", '?': "missing", 'u': "cup", 'z': "rhizomorphs"})
data['stalk_surface_above_ring'] = data['stalk_surface_above_ring'].map({'s': "smooth", 'f': "fibrous", 'k': "silky", 'y': "scaly"})
data['stalk_surface_below_ring'] = data['stalk_surface_below_ring'].map({'s': "smooth", 'f': "fibrous", 'y': "scaly", 'k': "silky"})
data['stalk_color_above_ring'] = data['stalk_color_above_ring'].map({'w': "white", 'g': "gray", 'p': "pink", 'n': "brown", 'b': "buff", 'e': "red", 'o': "orange", 'c': "cinnamon", 'y': "yellow"})
data['stalk_color_below_ring'] = data['stalk_color_below_ring'].map({'w': "white", 'p': "pink", 'g': "gray", 'b': "buff", 'n': "brown", 'e': "red", 'y': "yellow", 'o': "orange", 'c': "cinnamon"})
data['veil_type']= ['partial' if each == 'p' else 'universal' for each in data['veil_type']]
data['veil_color'] = data['veil_color'].map({'w': "white", 'n': "brown", 'o': "orange", 'y': "yellow"})
data['ring_number'] = data['ring_number'].map({'o': "one", 't': "two", 'n': "none"})
data['ring_type'] = data['ring_type'].map({'p': "pendant", 'e': "evanescent", 'l': "large", 'f': "flaring", 'n': "none", 's': "sheathing", 'z': "zone"})
data['spore_print_color'] = data['spore_print_color'].map({'k': "black", 'n': "brown", 'u': "purple", 'h': "chocolate", 'w': "white", 'r': "green", 'o': "orange", 'y': "yellow", 'b': "buff"})
data['population'] = data['population'].map({'s': "scattered", 'n': "numerous", 'a': "abundant", 'v': "several", 'y': "solitary", 'c': "clustered"})
data['habitat'] = data['habitat'].map({'u': "urban", 'g': "grasses", 'm': "meadows", 'd': "woods", 'p': "paths", 'w': "waste", 'l': "leaves"})


In [6]:
data.head()

,classes,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,poisonous,convex,smooth,brown,bruises,pungent,free,close,narrow,black,...,smooth,white,white,partial,white,one,pendant,black,scattered,urban
1,edible,convex,smooth,yellow,bruises,almond,free,close,broad,black,...,smooth,white,white,partial,white,one,pendant,brown,numerous,grasses
2,edible,bell,smooth,white,bruises,anise,free,close,broad,brown,...,smooth,white,white,partial,white,one,pendant,brown,numerous,meadows
3,poisonous,convex,scaly,white,bruises,pungent,free,close,narrow,brown,...,smooth,white,white,partial,white,one,pendant,black,scattered,urban
4,edible,convex,smooth,gray,no,none,free,crowded,broad,black,...,smooth,white,white,partial,white,one,evanescent,brown,abundant,grasses


In [7]:
le = LabelEncoder()

# Iterate over all the values of each column and extract their dtypes
for col in data:
    if data[col].dtype == 'object':
        data[col] = le.fit_transform(data[col])

In [8]:
data.head()

,classes,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,1,2,3,0,0,7,1,0,1,0,...,3,7,7,0,2,1,4,0,3,4
1,0,2,3,9,0,0,1,0,0,0,...,3,7,7,0,2,1,4,1,2,0
2,0,0,3,8,0,1,1,0,0,1,...,3,7,7,0,2,1,4,1,2,2
3,1,2,2,8,0,7,1,0,1,1,...,3,7,7,0,2,1,4,0,3,4
4,0,2,3,3,1,6,1,1,0,0,...,3,7,7,0,2,1,0,1,0,0


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   classes                   8124 non-null   int64
 1   cap_shape                 8124 non-null   int64
 2   cap_surface               8124 non-null   int64
 3   cap_color                 8124 non-null   int64
 4   bruises                   8124 non-null   int64
 5   odor                      8124 non-null   int64
 6   gill_attachment           8124 non-null   int64
 7   gill_spacing              8124 non-null   int64
 8   gill_size                 8124 non-null   int64
 9   gill_color                8124 non-null   int64
 10  stalk_shape               8124 non-null   int64
 11  stalk_root                8124 non-null   int64
 12  stalk_surface_above_ring  8124 non-null   int64
 13  stalk_surface_below_ring  8124 non-null   int64
 14  stalk_color_above_ring    8124 non-null 

In [10]:
data.describe()

,classes,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
count,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,...,8124.000000,8124.000000,8124.000000,8124.0,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000
mean,0.482029,2.491876,1.742984,4.323486,0.584441,4.788282,0.974151,0.161497,0.309207,4.729444,...,2.424914,5.446578,5.393402,0.0,1.965534,1.069424,2.291974,3.062038,3.644018,3.221073
std,0.499708,0.901287,1.179629,3.444391,0.492848,1.983678,0.158695,0.368011,0.462195,3.342402,...,0.870347,2.143900,2.194604,0.0,0.242669,0.271064,1.801672,2.825308,1.252082,2.530692
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,0.000000,0.000000,4.000000,1.000000,0.000000,0.000000,2.000000,...,2.000000,5.000000,5.000000,0.0,2.000000,1.000000,0.000000,1.000000,3.000000,0.000000
50%,0.000000,2.000000,2.000000,3.000000,1.000000,6.000000,1.000000,0.000000,0.000000,4.000000,...,3.000000,7.000000,7.000000,0.0,2.000000,1.000000,2.000000,3.000000,4.000000,3.000000
75%,1.000000,3.000000,3.000000,8.000000,1.000000,6.000000,1.000000,0.000000,1.000000,7.000000,...,3.000000,7.000000,7.000000,0.0,2.000000,1.000000,4.000000,7.000000,4.000000,6.000000
max,1.000000,5.000000,3.000000,9.000000,1.000000,8.000000,1.000000,1.000000,1.000000,11.000000,...,3.000000,8.000000,8.000000,0.0,3.000000,2.000000,4.000000,8.000000,5.000000,6.000000


In [11]:
# Creating the target variable for the test and training sets of data
X = data.drop('classes', axis = 1)
y = data["classes"]

# Split the dataset into training and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [12]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [13]:
# logreg to predict instances from the test set and store it
y_pred = model.predict(X_test)

# Printing the confusion matrix of the logreg model
confusion_matrix(y_test, y_pred)

array([[1257,    0],
       [   0, 1181]])

In [14]:
# import scikitplot as skplt
# import matplotlib.pyplot as plt

# # y_true = # ground truth labels
# # y_probas = # predicted probabilities generated by sklearn classifier
# skplt.metrics.plot_roc_curve(y_test, y_pred)
# plt.show()